In [60]:
import pandas as pd

# Cargar los datos
invoice_header = pd.read_csv('/content/invoices_header.csv', sep = ";")
invoice_products = pd.read_csv('/content/invoices_products.csv', sep = ";")
products = pd.read_csv('/content/products.csv', sep = ";")
suppliers = pd.read_csv('/content/suppliers.csv', sep = ";")
daily_currencies = pd.read_csv('/content/daily_currencies.csv', sep = ";")

In [38]:
print("invoice_header\n",invoice_header.info())
print("invoice_products\n",invoice_products.info())
print("products\n",products.info())
print("suppliers\n",suppliers.info())
print("daily_currencies\n",daily_currencies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5178 entries, 0 to 5177
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      5178 non-null   object        
 1   InboundDate  5178 non-null   datetime64[ns]
 2   Supplier     5178 non-null   object        
 3   OrderDate    5178 non-null   datetime64[ns]
 4   InvoiceDate  5178 non-null   datetime64[ns]
dtypes: datetime64[ns](3), object(2)
memory usage: 202.4+ KB
invoice_header
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39823 entries, 0 to 39822
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Invoice               39823 non-null  object 
 1   Quantity              39823 non-null  int64  
 2   Product               39823 non-null  object 
 3   PurchasePrice (Unit)  39823 non-null  float64
 4   Section               39823 non-null  object 
dtyp

In [61]:
# Limpiar y transformar los datos
# Convertir fechas a datetime
invoice_header['OrderDate'] = pd.to_datetime(invoice_header['OrderDate'])
invoice_header['InboundDate'] = pd.to_datetime(invoice_header['InboundDate'])
invoice_header['InvoiceDate'] = pd.to_datetime(invoice_header['InvoiceDate'])
daily_currencies['Date'] = pd.to_datetime(daily_currencies['Date'])

# Merge de datos para obtener información completa de las facturas
invoice_data = pd.merge(invoice_products, invoice_header, on='Invoice')
invoice_data = pd.merge(invoice_data, products, on='Product')
invoice_data = pd.merge(invoice_data, suppliers, left_on='Supplier',right_on='IDSupplier')

invoice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39823 entries, 0 to 39822
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Invoice               39823 non-null  object        
 1   Quantity              39823 non-null  int64         
 2   Product               39823 non-null  object        
 3   PurchasePrice (Unit)  39823 non-null  float64       
 4   Section               39823 non-null  object        
 5   InboundDate           39823 non-null  datetime64[ns]
 6   Supplier              39823 non-null  object        
 7   OrderDate             39823 non-null  datetime64[ns]
 8   InvoiceDate           39823 non-null  datetime64[ns]
 9   Type                  30158 non-null  object        
 10  Division              39823 non-null  object        
 11  Group                 39823 non-null  object        
 12  ShortDescription      38673 non-null  object        
 13  Description     

In [62]:
# Calcular el importe en EUR (al cierre diario del valor de la divisa de esa fecha en Euros)
invoice_data = pd.merge(invoice_data, daily_currencies, left_on=['Currency', 'InvoiceDate'], right_on=['Currency', 'Date'])
invoice_data['amount_eur'] = invoice_data['PurchasePrice (Unit)'] * invoice_data['Quantity'] * invoice_data['Close']

# Calcular Lead Time real
invoice_data['lead_time_real'] = (invoice_data['InboundDate'] - invoice_data['OrderDate']).dt.days

# Calcular Lead Time teórico
def calculate_theoretical_lead_time(row):
    if row['Country'] == 'Spain':
        return 10
    elif row['Currency'] == 'EUR':
        return 20
    else:
        return 45

invoice_data['lead_time_theoretical'] = invoice_data.apply(calculate_theoretical_lead_time, axis=1)

invoice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18003 entries, 0 to 18002
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Invoice                18003 non-null  object        
 1   Quantity               18003 non-null  int64         
 2   Product                18003 non-null  object        
 3   PurchasePrice (Unit)   18003 non-null  float64       
 4   Section                18003 non-null  object        
 5   InboundDate            18003 non-null  datetime64[ns]
 6   Supplier               18003 non-null  object        
 7   OrderDate              18003 non-null  datetime64[ns]
 8   InvoiceDate            18003 non-null  datetime64[ns]
 9   Type                   17009 non-null  object        
 10  Division               18003 non-null  object        
 11  Group                  18003 non-null  object        
 12  ShortDescription       18003 non-null  object        
 13  D

In [65]:
# Agregar datos para obtener métricas
# Importe total de compras diarias por sección
daily_purchases = invoice_data.groupby(['InvoiceDate', 'Section'])['amount_eur'].sum().reset_index()

# Ranking de proveedores por importe total comprado y número de productos distintos
supplier_ranking = invoice_data.groupby('IDSupplier').agg({
    'amount_eur': 'sum',
    'Product': 'nunique'
}).reset_index()

# Media anual de lead time por producto-proveedor
invoice_data['order_year'] = invoice_data['OrderDate'].dt.year
annual_lead_time = invoice_data.groupby(['IDSupplier', 'Product', 'order_year']).agg({
    'lead_time_real': 'mean',
    'lead_time_theoretical': 'mean'
}).reset_index()

# Visualizamos tablas resultantes
print(daily_purchases.sort_values(["InvoiceDate","Section"],ascending=[False,True]))
print(round(supplier_ranking.sort_values("amount_eur",ascending=False),1))
print(annual_lead_time.head())

     InvoiceDate    Section   amount_eur
1373  2021-05-21  Seccion A  3595.691201
1374  2021-05-21  Seccion B  3467.590734
1375  2021-05-21  Seccion C  4029.357591
1376  2021-05-21  Seccion D  2791.534816
1377  2021-05-21  Seccion E  3228.429271
...          ...        ...          ...
7     2014-05-19  Seccion E  2927.872992
8     2014-05-19  Seccion F  4672.329791
0     2014-05-15  Seccion C  1393.276726
1     2014-05-15  Seccion D   103.260283
2     2014-05-15  Seccion E  1242.041522

[1379 rows x 3 columns]
   IDSupplier  amount_eur  Product
11   PROV1585  10250625.9      614
7    PROV1475   5825498.1      250
4    PROV1398   2099955.9      189
24   PROV1976   1092480.6      559
3    PROV1397    836788.8       40
1    PROV1061    699134.6      641
17   PROV1816    494450.6       31
12   PROV1597    455950.7       99
15   PROV1756    440016.2      124
19   PROV1875    435026.7      412
18   PROV1834    323803.0      140
28   PROV2197    289484.9      211
20   PROV1936    256034.8   

In [83]:
# Guardar las tablas del datamart en ficheros
daily_purchases.to_csv('daily_purchases.csv', index=False)
supplier_ranking.to_csv('supplier_ranking.csv', index=False)
annual_lead_time.to_csv('annual_lead_time.csv', index=False)

,Año,Mes,Seccion A,Seccion B,Seccion C,Seccion D,Seccion E,Seccion F
0,2016,enero,42945.337028,290876.303302,100999.634234,112532.932235,141252.738732,37593.612234
1,2016,febrero,81076.768475,88761.456577,141404.06545,93528.930214,113560.263801,161779.622336
2,2016,marzo,23948.847109,130008.956158,71898.172019,191529.880689,134844.281638,54922.159147
3,2016,abril,56875.278303,69572.415457,983552.333135,119788.879017,-5762369.360581,91795.698122
4,2016,mayo,53889.734025,113079.362842,74715.450382,110633.91369,147270.632915,61122.571929


In [111]:
# Opcional: Incorporar análisis mensualizado en comparación con el presupuesto
purchase_budget = (
    pd.read_excel('purchase_budget.xls')
    .dropna(how = "all")      # No esta centrada la tabla en el fichero
    .reset_index(drop=True)   # Para que inicie el indice de nuevo en la primera fila
    .iloc[:, 2:]              # Le obligo a ignorar las primeras dos columnas (vacias)
)

purchase_budget.columns = purchase_budget.iloc[0]                   # Primera fila es el cabecero
purchase_budget = purchase_budget.iloc[1:].reset_index(drop=True)   # Nos cargamos la primera fila ya redundante

print(purchase_budget.head())

invoice_data['invoice_month'] = invoice_data['InvoiceDate'].dt.to_period('M')
monthly_purchases = invoice_data.groupby(['invoice_month', 'Section'])['amount_eur'].sum().reset_index()
print("Compras mensuales por sección:\n",round(monthly_purchases.head(),2))

# El mes está en formato mes datetime, hay que convertir alguno de los dos al formato del otro

0   Año      Mes     Seccion A      Seccion B      Seccion C      Seccion D  \
0  2016    enero  42945.337028  290876.303302  100999.634234  112532.932235   
1  2016  febrero  81076.768475   88761.456577   141404.06545   93528.930214   
2  2016    marzo  23948.847109  130008.956158   71898.172019  191529.880689   
3  2016    abril  56875.278303   69572.415457  983552.333135  119788.879017   
4  2016     mayo  53889.734025  113079.362842   74715.450382   110633.91369   

0       Seccion E      Seccion F  
0   141252.738732   37593.612234  
1   113560.263801  161779.622336  
2   134844.281638   54922.159147  
3 -5762369.360581   91795.698122  
4   147270.632915   61122.571929  
Compras mensuales por sección:
   invoice_month    Section  amount_eur
0       2014-05  Seccion A      927.69
1       2014-05  Seccion B     3758.50
2       2014-05  Seccion C     2854.17
3       2014-05  Seccion D     1010.09
4       2014-05  Seccion E     4169.91


In [112]:
# Creamos un diccionario de meses en español, para transformarlos a formato numérico
month_dict = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',
    'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

# Convertir la columna 'Mes' a su formato numérico
purchase_budget['Mes_Num'] = purchase_budget['Mes'].map(month_dict)

# Creamos la columna 'Fecha' combinando 'Año' y 'Mes_Num' y convirtiéndola a datetime
purchase_budget['Fecha'] = pd.to_datetime(
    purchase_budget['Año'].astype(str) + '-' + purchase_budget['Mes_Num'],
    format='%Y-%m'
).dt.to_period('M')

purchase_budget.info() # Las columnas son str, debemos convertir a float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype    
---  ------     --------------  -----    
 0   Año        72 non-null     object   
 1   Mes        72 non-null     object   
 2   Seccion A  72 non-null     object   
 3   Seccion B  72 non-null     object   
 4   Seccion C  72 non-null     object   
 5   Seccion D  72 non-null     object   
 6   Seccion E  72 non-null     object   
 7   Seccion F  72 non-null     object   
 8   Mes_Num    72 non-null     object   
 9   Fecha      72 non-null     period[M]
dtypes: object(9), period[M](1)
memory usage: 5.8+ KB


In [113]:
print(purchase_budget.columns)
purchase_budget[['Seccion A', 'Seccion B', 'Seccion C', 'Seccion D','Seccion E', 'Seccion F']] = purchase_budget[['Seccion A', 'Seccion B', 'Seccion C', 'Seccion D','Seccion E', 'Seccion F']].apply(pd.to_numeric)

purchase_budget = purchase_budget.drop(["Año","Mes","Mes_Num"], axis = 1)
print(round(purchase_budget.head(),2))
# Esta en formato incorrecto, debemos pasar las secciones de columnas a filas

Index(['Año', 'Mes', 'Seccion A', 'Seccion B', 'Seccion C', 'Seccion D',
       'Seccion E', 'Seccion F', 'Mes_Num', 'Fecha'],
      dtype='object', name=0)
0  Seccion A  Seccion B  Seccion C  Seccion D   Seccion E  Seccion F    Fecha
0   42945.34  290876.30  100999.63  112532.93   141252.74   37593.61  2016-01
1   81076.77   88761.46  141404.07   93528.93   113560.26  161779.62  2016-02
2   23948.85  130008.96   71898.17  191529.88   134844.28   54922.16  2016-03
3   56875.28   69572.42  983552.33  119788.88 -5762369.36   91795.70  2016-04
4   53889.73  113079.36   74715.45  110633.91   147270.63   61122.57  2016-05


In [118]:
purchase_budget_melted = purchase_budget.melt(id_vars=['Fecha'], var_name='Section', value_name='Budget')
purchase_budget_melted = purchase_budget_melted.sort_values(["Fecha","Section"]).head()
print("Presupuesto mensual por sección:\n",round(purchase_budget_melted.head(),2))
print("Compras mensuales por sección:\n",round(monthly_purchases.head(),2))

# Corregido!

Presupuesto mensual por sección:
        Fecha    Section     Budget
0    2016-01  Seccion A   42945.34
72   2016-01  Seccion B  290876.30
144  2016-01  Seccion C  100999.63
216  2016-01  Seccion D  112532.93
288  2016-01  Seccion E  141252.74
Compras mensuales por sección:
   invoice_month    Section  amount_eur
0       2014-05  Seccion A      927.69
1       2014-05  Seccion B     3758.50
2       2014-05  Seccion C     2854.17
3       2014-05  Seccion D     1010.09
4       2014-05  Seccion E     4169.91


In [123]:
budget_comparison = pd.merge(monthly_purchases, purchase_budget_melted, left_on=['invoice_month', 'Section'], right_on=['Fecha', 'Section'], how='inner')
budget_comparison["Spenditure"] = round(budget_comparison["Budget"] - budget_comparison["amount_eur"],2)
print(budget_comparison.head())
budget_comparison.to_csv('budget_comparison.csv', index=False)